## From Q-Learning to Proximal Policy Optimization

## 1. From Q-Learning to Policy Gradients 

In order to get the actions with different states, we want to find a function would approaximate the **excepted value** for each (action, state) pair, means $q(a, s)$ function. 

There are two kinds of methods to get the $q(a, s)$, one based on the whole episode and using Bellman function, and another uses one-step (Temporal Difference )to converge to $q^*(a, s)$. As we get the $q(a, s)$, we could use greedy or $\epsilon$-greedy methods get each action when the agent faces different states.

However, there are some shortages when we get $q(a, s)$ firstly, and use its value get *best* or *$\epsilon$-best* actions. 

1. WITHOUT REAL STOCHASTIC: There are so many occasions, such like an imperfectable information game, the best policy may be **stochastic**, we need to get the precisely probabiliy of each action, not just by greedy or $\epsilon$-greedy. 

2. BRITTLE: $\epsilon$-greedy selection may change dramatically for an arbitrarily small change in the estimated action values. 

3. HARD FOR CONTINUOUS: the $q(a, s)$ function is hard for continuous space and action problem. 

4. INFORMACTION IMPERFECT: because of our sensor limitation, there will be some situations that looks same, but actually are different. In such situations, we need different actions when face the *same* situations. 

Therefore, we need a methods which could get the probability of given **state-action** pair. For continous space, we could get the value of each index or categorical of actions. 

Under the policy $\pi(\theta)$, for an episode, we want maxmize the **value** of **initial state** $s_0$, if we define the $J(\theta) = v_{\pi_{\theta}(s_0)} $, our target is:
$$argmax_{\theta}{J(\theta)} $$

we use the **tracjectories** to be a finite episode or continuous episode with horizon H. If we collect so many tracjectories $(\tau^{0},\tau^{1}, \tau^{2},..., \tau^{N})$

Therefore, for these tracjectories $J(\theta) = \sum_{\tau}Pr(\tau; \theta)R(\theta) $

$$ 
\begin{align}
\nabla {J(\theta)} &= \nabla {\sum_{\tau} Pr(\tau; \theta) R(\tau)} \\
&=   {\sum_{\tau} \nabla {Pr(\tau; \theta) R(\tau)}} \\
&=   {\sum_{\tau} \frac{Pr(\tau; \theta)} {Pr(\tau; \theta)} \nabla {Pr(\tau; \theta) R(\tau)}} \\
&= \sum_{\tau} Pr(\tau; \theta) \frac {\nabla_{\theta} Pr(\tau; \theta)} {Pr(\tau; \theta)} R(\tau) \\
&= \frac{1}{m} \sum_{i = 0 }^{m}\nabla_{\theta} In(Pr(\tau; \theta))R(\tau^i) \text{  ; for sampled m tracjectories}  \\ 
&= \frac{1}{m} \sum_{i = 0 }^{m} \nabla_{\theta} In(\prod_{t=0}^{H} Pr(s_{t+1}^{i}| s_{t}^i, a_t^i) \pi_{\theta}(a_t^i | s_{t}^i)) R(\tau^i)\\
&= \frac{1}{m} \sum_{i = 0 }^{m} \nabla_{\theta} \left[{\sum_{t = 0}^{H-1} In(Pr(s_{t+1}^i | s_t^i, a_t^i)) + \sum_{i = 0}^{H} In(\pi_{\theta}(a_t^i | s_t^i))}]R(\tau^i) \right]\\
&= \frac{1}{m} \sum_{i = 0 }^{m} \nabla_{\theta} \left[ \sum_{t = 0}^{H-1} In(\pi_{\theta}(a_t^i | s_t^i))]R(\tau^i) \right]  \\
&= \frac{1}{m} \sum_{i = 0 }^{m}\sum_{t = 0}^{H-1} \nabla_{\theta}In(\pi_{\theta}(a_t^i | s_t^i))R(\tau^i) \\
&\propto \frac{1}{m} \sum_{i = 0 }^{m}\sum_{t = 0}^{H-1} \nabla_{\theta}log(\pi_{\theta}(a_t^i | s_t^i))R(\tau^i)
\end{align}
$$

define the $\nabla{J(\theta)}$ as $\hat{g}$, and for each step $t$, we approximate the reward $R(\tau)$ as the **future reward** $R(\tau^t)$, for a given sample, we get: 

 $$ \hat{g} = \sum_{t = 0}^{H-1} \nabla_{\theta}log(\pi_{\theta}(a_t^i | s_t^i))\sum_{t=t}^{H}r(s_{i, t}, a_{i, t})$$

$$ \mathbf{\theta} = \theta +  \alpha \hat{g} $$

Therefore, we could get sample ${s_i, a_i}$ for $\pi(a | s)$: 

1. get the rewards $\sum_{t=t}^{H}r(s_{i, t}, a_{i, t})$
2. evalutate $\nabla_{\theta}{J(\theta)} = \nabla_{\theta}log(\pi_{\theta}(a_t^i | s_t^i))\sum_{t=t}^{H}r(s_{i, t}, a_{i, t})$
3. $\theta \leftarrow \theta + \alpha \nabla J_\theta (\theta)$

## 2.  From Policy Gradients to Actor-Critic

We call the previous algorithm **REINFORCE**. The algorithm could start an arbitary $\pi(\theta) $, and based on the update, get better $\theta$. 

But, the REINFORCE methods have high **variance**. 

why?

because when we get the $\theta$, we just could get limited actions from limited spaces. We could add some **baseline** to reduce the variance. 

If we use the value function **$v(s)$** to be the baseline, we could: 

1. get the action policy $\pi_\theta(\theta)$, we called actor
2. get the policy evaluation method $v_{w}(w)$, we called critic

because of the $v(s)$ function, we could change our algorithm to an online algorithm, which doesn't need run an episode to get the rewards. 


**online actor-critic algorithm**

1. take action$ a \sim \pi_{\theta}(a | s) $, get $(s, a, s', r)$
2. evaluate $\hat{A}^{\pi}(s, a) = r(s, a) + \gamma v_{w}^{\pi}(s') - v_{w}^{\pi}(s)$
3. $ w \leftarrow  \hat{A}^{\pi}(s, a) \nabla{V}(s, w) $
4. $ \nabla J(\theta) = \nabla log \pi_{\theta}(a | s) \hat{A^{\pi}}(s, a) $
5. $ \theta \leftarrow \theta + \alpha \nabla_{\theta}J(\theta)$


We could add some methods to imporve the performance of this algorithm: 

1. N-step bootstrap
2. Generalized Advantrage Estimation (GAE)
3. A2C and A3C methods

## 3. TRPO and PPO

When we use the REINFORCE or Actor-Critic methods, we get a sample and use it and drop out it. What if we can use the tracjectories that we collected over and over again, and import our policy? 

Remember when we do policy gradient, we want to maximize the $J(\theta) = \mathbb{E}_{(\tau; s_0, a_0, .. )} [\sum_{t = 0} ^ {\infty} \gamma^t r(s_t)]$

If there are a new policy $\pi(\theta')$ , we take the actions are sampled from $\pi(\theta')$  to get tracjectory $\tau $

$$ 
\begin{align}
\because A_{\pi{\theta}} &= {E}_{s' \sim Pr(s' | s, a)}[r(s) + \gamma V_{\pi(\theta)}(s') - V_{\pi(\theta)}(s)] \\
\therefore E_{\tau | \pi(\theta')}[\sum_{t = 0} ^ {\infty} \gamma^{t} A_{\pi{\theta}}] &= E_{\tau | \pi(\theta')} [ \sum_{t = 0}^{\infty} \gamma^{t}(r(s) + \gamma V_{\pi(\theta)}(s') - V_{\pi(\theta)}(s))] \\
&= E_{\tau | \pi(\theta')} \left[ \sum_{t = 0}^{\infty} (\gamma^{t} r(s) + [\gamma^t * \gamma V_{\pi(\theta)}(s_{t+1)}) - \gamma^t V_{\pi}(s_t))]) \right] \\
&=E_{\tau | \pi(\theta')} \left[ \sum_{t = 0}^{\infty} (\gamma^{t} r(s)) - V_{\pi(\theta)}(s_0) \right] \\
&=J(\theta') - E_{\tau | \pi(\theta')}V_{\pi(\theta)}(s_0)\\
&=J(\theta') - E_{s_o}[V_{\pi(\theta)}(s_0)] \text{; because distribute $s_0$ is independent of $\theta$}\\
&=J(\theta') - J(\theta)
\end{align}
$$

Rewrite it as: 

$$ J(\theta') - J(\theta) = E_{\tau | \pi(\theta')}[\sum_{t = 0} ^ {\infty} \gamma^{t} A_{\pi{\theta}}] $$

Because the $\tau$ is consist on (action, state), based on the margin distribution

$$
\begin{align}
J(\theta') - J(\theta) &= E_{\tau | \pi(\theta')}[\sum_t \gamma^t A_{\pi{\theta}}]  \\
&= \sum_{t} E_{s_t \sim p(\theta)}  [E_{a_t \sim \pi(\theta'(a_t | s_t))}[\gamma^{t} A_{\pi{\theta}} (s_t, a_t)]]
\end{align}
$$

based on the sample importance: 

$$\begin{align}
E_{x \sim p(x) [f(x)]} &= \int p(x) f(x) dx \\
&= \int \frac{q(x)}{q(x)} p(x) f(x)dx \\
&= \int q(x)\frac{p(x)}{q(x)}f(x) dx \\
&= E_{x \sim q(x)} \left[\frac{p(x)}{q(x)}f(x) \right]
\end{align}$$

$$\begin{align}
J(\theta') - J(\theta) &= E_{\tau | \pi(\theta')}[\sum_t \gamma^t A_{\pi{\theta}}]  \\
&= \sum_{t} E_{s_t \sim p(\theta')}  [E_{a_t \sim \pi(\theta'(a_t | s_t))}[\gamma^{t} A_{\pi{\theta}} (s_t, a_t)]] \\
&= \sum_{t} E_{s_t \sim p(\theta')}  \left[ E_{a_t \sim \pi_{\theta}}(a_t | s_t) \left[ \frac{\pi_{\theta'}(a_t, s_t)}{\pi_{\theta}(a_t, s_t)} \gamma^{t} A_{\pi{\theta}} (s_t, a_t) \right] \right]
\end{align}$$

And, we know the $\pi(\theta)$, therefore, we could get the $E_{a_t \sim \pi_{ (\theta) } (a_t | s_t)}$ value

current now, we did not know the $E_{s_t \sim p(\theta')}$, because the $\theta'$ is the new policy paramters we want get. 

However, if we could approaximate the $p(\theta')$ using $\theta$, we could update the $J(\theta)$ to $J(\theta')$ over and over again, just using the $\theta$, which we already know, and tracjectories we already have collected. 

It is to say, if we could make fellow be true: 
$$\begin{align}
J(\theta') - J(\theta) &= \sum_{t} E_{s_t \sim p(\theta')}  \left[ E_{a_t \sim \pi_{\theta}}(a_t | s_t) \left[ \frac{\pi_{\theta'}(a_t, s_t)}{\pi_{\theta}(a_t, s_t)} \gamma^{t} A_{\pi{\theta}} (s_t, a_t) \right] \right] \\
&\approx \sum_{t} E_{s_t \sim p(\theta)}  \left[ E_{a_t \sim \pi_{\theta}}(a_t | s_t) \left[ \frac{\pi_{\theta'}(a_t, s_t)}{\pi_{\theta}(a_t, s_t)} \gamma^{t} A_{\pi{\theta}} (s_t, a_t) \right] \right] 
\end{align}$$
if we define $\bar{A}(\theta') = \sum_{t} E_{s_t \sim p(\theta)}  \left[ E_{a_t \sim \pi_{\theta}}(a_t | s_t) \left[ \frac{\pi_{\theta'}(a_t, s_t)}{\pi_{\theta}(a_t, s_t)} \gamma^{t} A_{\pi{\theta}} (s_t, a_t) \right] \right]  $ 

We could use the: 

$$\theta' \leftarrow argmax_{\theta'} \bar{A}(\theta')$$

to update $J(\theta)$ as quick as possible: 

$$ J(\theta') - J(\theta) \approx \bar{A}(\theta')$$

There brust out a question, when $E_{s_t \sim p(\theta')} \approx E_{s_t \sim p(\theta')}$

### The bounds of constraints

Case1: if $\pi_(\theta)$ is a deterministic policy s.t $a_t = \pi_\theta(s_t)$

for a policy $\pi'$, which gives $a_t'$, define $P(a \neq a') \leq \epsilon$ for all states

$$\begin{align} 
P_{\theta'}(s_t) &= (1 - \epsilon)^t P_{\theta} (s_t) + (1 - (1 - \epsilon)^t) P_{diff}(s_t) \\
|P_{\theta'}(s_t) - p_{\theta}(s_t)| &= (1 - (1 - \epsilon)^t) |P_{diff}(s_t) - p_{\theta}(s_t)| \leq 2(1 - (1 - \epsilon)^t) \leq 2 \epsilon t
\end{align}$$

Case2: $\pi(\theta)$ is a distribution

for a policy $\pi'$, which gives $a_t'$, define the two policy are **close** if $ |\pi_{\theta'}(a_t | s_t) - \pi_{\theta}(a_t | s_t)| \leq \epsilon$ for all states


$$\begin{align} 
E_{p_{\theta'}(s_t)} [f(s_t)]= \sum_{s_t} p_{\theta'}(s_t)f(s_t) & \geq \sum_{s_t} \left[  p_\theta{s_t} - |p_{\theta} - p_{\theta'}(s_t)| max_{s_t}f(s_t)  \right] \\
&\geq E_{p_{\theta}(s_t)} \left[ f(s_t) \right] - 2\epsilon t max_{s_t} f(s_t) 
\end{align}$$


We can get $J(\theta')$ with sampled importance: 

$$\begin{align}
&\sum_{t} E_{s_t \sim p(\theta')}  \left[ E_{a_t \sim \pi_{\theta}}(a_t | s_t) \left[ \frac{\pi_{\theta'}(a_t, s_t)}{\pi_{\theta}(a_t, s_t)} \gamma^{t} A_{\pi{\theta}} (s_t, a_t) \right] \right] \geq \\
&\sum_{t} E_{s_t \sim p(\theta)}  \left[ E_{a_t \sim \pi_{\theta}}(a_t | s_t) \left[ \frac{\pi_{\theta'}(a_t, s_t)}{\pi_{\theta}(a_t, s_t)} \gamma^{t} A_{\pi{\theta}} (s_t, a_t) \right] \right] - \sum_{t} 2 \epsilon t C
\end{align}$$

And the $C$ in above equation is the accumulated excepted rewards for all steps.

$$ C \sim O(\frac{r_{max}}{ 1 - \gamma}) $$

Current now, we get the bound that could use the previously collected tracjectories to update $\theta$ to $\theta'$ 

### $D_{TV}$ to $D_{KL}$

based on the Pinsker's inequality, for p, q two probability distributions, then: 

$$ \delta(p, q) \leq \sqrt{(\frac{1}{2} D_{KL}(p | q))} \text{  , with $D_{KL}(p | q) = \sum_{i \in X} \left( log \frac{p(i)} {q(i)} p(i)\right)$} $$

We could get the $|\pi(\theta) - \pi(\theta')| \leq \sqrt{\frac{1}{2} D_{KL} (\pi(\theta) | \pi(\theta'))}$

With the previous information, if we could define a small **$\epsilon$**  and find the $\theta'$ could satisfy following, we could update $\theta$ to $\theta'$ as fast as possible and keep the approximation. 

1. $\theta'$ makes $\sqrt{\frac{1}{2} D_{KL} (\pi(\theta) | \pi(\theta') } \leq \epsilon$ to be true, which makes sure the approximate of two policy

2. $\theta'$ make $ \sum_{t} E_{s_t \sim p(\theta)}  \left[ E_{a_t \sim \pi_{\theta}}(a_t | s_t) \left[ \frac{\pi_{\theta'}(a_t, s_t)}{\pi_{\theta}(a_t, s_t)} \gamma^{t} A_{\pi{\theta}} (s_t, a_t) \right] \right] $ as large as possible, which could make $\theta$ update to $\theta'$ as fast as possible

We call the $D_{KL}(\pi_{\theta'}(a_t | s_t)|| (\pi_{\theta}(a_t | s_t)) \leq \epsilon$ **TRUST REGION**

Such as, We could using the following Loss function to satisfy the above two constrains: 

$$ \mathcal{L}(\theta', \lambda) =  \sum_{t} E_{s_t \sim p(\theta)}  \left[ E_{a_t \sim \pi_{\theta}}(a_t | s_t) \left[ \frac{\pi_{\theta'}(a_t, s_t)}{\pi_{\theta}(a_t, s_t)} \gamma^{t} A_{\pi{\theta}} (s_t, a_t) \right] \right]  - \lambda (D_{KL}(\pi_{\theta'}(a_t | s_t)|| (\pi_{\theta}(a_t | s_t))) - \epsilon)$$

Updaing as the iteration: 

1. Maximize $\mathcal{L}$ w.s.t to $\theta$
2. $\lambda \leftarrow \lambda  + \alpha \lambda (D_{KL}(\pi_{\theta'}(a_t | s_t)|| (\pi_{\theta}(a_t | s_t))) - \epsilon) $

We call the $\mathcal{L}(\theta') = \sum_{t} E_{s_t \sim p(\theta)}  \left[ E_{a_t \sim \pi_{\theta}}(a_t | s_t) \left[ \frac{\pi_{\theta'}(a_t, s_t)}{\pi_{\theta}(a_t, s_t)} \gamma^{t} A_{\pi{\theta}} (s_t, a_t) \right] \right]  $ **surrgate loss function. **

Based on the Taylor Approaximation, we could get: 

$$\begin{align} \mathcal{L}_{\theta'} &\approx \mathcal{L}_{\theta} + \nabla_{\theta'} \mathcal{L}_{\theta}(\theta')(\theta' - \theta) \\
D_{KL}(\theta' || \theta) &\approx \frac{1}{2} (\theta' - \theta)^T H(\theta' - \theta)
\approx \frac{1}{2} \| \theta - \theta'\|^2 \end{align}$$ 



We can get the solution of this: 

$$ \theta' = \theta + \sqrt{\frac{\epsilon}{\| \nabla_{\theta}  \mathcal{L(\theta)} ^ 2\|}}  \nabla_{\theta}  \mathcal{L(\theta)} $$

#### Trust Region Policy Optimization Algorithm 

There are some problems with above update. 
1. Might not to robutst to turst region size $\epsilon$; at some iteration $\epsilon$ may be too large and performance and degrade
2. Because of quadratic qpproximate, KL-divergence constraint may be violated 

Solution: 
1. Require imporovement in surrogate (make sure that $\mathcal{L}_{\theta}(\theta') \geq 0$)
2. Enforce KL-constraint

1. Compute proposed policy $\Delta_{k} = \sqrt{\frac{\epsilon}{\| \nabla_{\theta}  \mathcal{L(\theta)} ^ 2\|}}  \nabla_{\theta}  \mathcal{L(\theta)}$
2. for j = 0, 1, 2, ... , H, do:
   > if $ L_(\theta)(\theta')\geq 0 $ and $D_{KL}(\theta | \theta') \leq \delta$:
   > > accept the update and set $ \theta' = \theta + \alpha^j \Delta_{k} $

## Proximal Policy Optimization 

When using $\mathcal{L}(\theta') = E \left[ \frac{\pi_{\theta'}(a_t, s_t)} {\pi_{\theta}(a_t, s_t)} A_{\pi{\theta}} (s_t, a_t) \right]$ for one step $t$, 

the $\frac{\pi_{\theta'}(a_t, s_t)} {\pi_{\theta}(a_t, s_t)} $ may to large and get the update too dramatically.  In order to solve this problem, the paper proposed two methods to solve this problem: 

### PPO-Clipped Surrogate Objective 

This methods change the $\mathcal{L}$ to a clipped loss function.  let $r_{t}(\theta) = \frac{\pi_{\theta'}(a_t, s_t)} {\pi_{\theta}(a_t, s_t)}$:

$$\mathcal{L}^{CLIP}(\theta) = \mathbb{E}_t\left[ min(r_{t}(\theta), clip(r_t{\theta}, 1 - \epsilon, 1 + \epsilon)\bar{A_t})\right]$$

This method could let the updating not to small or to large. Could make the converagence more stable. 

### Adaptive KL Penalty Coefficient 

Beside clip the loss function, another way is using **Adaptive KL penalty Coefficient**

$$ \mathcal{L}^{KLPEN}(\theta) = \mathbb{E}_t  \left[ \frac{\pi_{\theta'}(a_t, s_t)} {\pi_{\theta}(a_t, s_t)} A_{t, \pi{\theta}}\right] - \beta D_{KL} \left[ \pi_{\theta}{(\cdot | s_t)}, \pi_{\theta'}{(\cdot | s_t)}\right]$$

Compute $d = \mathbb{E}_t D_{KL} \left[ \pi_{\theta}{(\cdot | s_t)}, \pi_{\theta'}{(\cdot | s_t)} \right]$

- if $ d < d_{targ} / 1.5, \beta \leftarrow \beta / 2$
- if $ d > d_{targ} * 1.5, \beta \leftarrow \beta * 2$

Based on the dynamic $\beta$ change, we could ensure the $D_{KL}$ to a suitable value. Which make the update not  too small or not too large.

## Conslusion

Based on the TRPO and PPO methods, especially PPO, we could using the tracjectories more efficiently, and update the $\theta$ to $\theta'$ as fast as possible and make sure the stable when update. 

## References

1. Trust region Policy Optimization: [https://paperswithcode.com/method/trpo]
2. Proximal Policy Optimization: [https://paperswithcode.com/method/ppo]
3. berkeley CS294: https://www.youtube.com/watch?v=ycCtmp4hcUs&list=PLkFD6_40KJIznC9CDbVTjAF2oyt8_VAe3&index=14
4. berkeley CS285: https://www.youtube.com/watch?v=QWnpF0FaKL4&list=PL_iWQOsE6TfURIIhCrlt-wj9ByIVpbfGc&index=41
5. Reinforcement Learning An Intorduction, Sutton